**Import Libraries**

In [1]:
import tensorflow as tf
from tensorflow.keras import layers, models
from tensorflow.keras.preprocessing.image import ImageDataGenerator
import numpy as np


**Load and Preprocess MNIST Data**

In [2]:
def load_and_preprocess_data():
    (x_train, y_train), (x_test, y_test) = tf.keras.datasets.mnist.load_data()

    # Normalize and expand dimensions
    x_train = x_train.astype('float32') / 255.0
    x_test = x_test.astype('float32') / 255.0

    # Add channel dimension (28x28 -> 28x28x1)
    x_train = np.expand_dims(x_train, axis=-1)
    x_test = np.expand_dims(x_test, axis=-1)

    # Convert labels to one-hot encoding
    y_train = tf.keras.utils.to_categorical(y_train, 10)
    y_test = tf.keras.utils.to_categorical(y_test, 10)

    return x_train, y_train, x_test, y_test

x_train, y_train, x_test, y_test = load_and_preprocess_data()


11490434/11490434 ━━━━━━━━━━━━━━━━━━━━ 0s 0us/step


**Create the CNN Model**

In [3]:
def build_cnn_model(num_filters1, num_filters2, kernel_size, dense_units):
    model = models.Sequential([
        layers.Conv2D(num_filters1, kernel_size, activation='relu', input_shape=(28, 28, 1)),
        layers.MaxPooling2D(pool_size=(2, 2)),
        layers.Conv2D(num_filters2, kernel_size, activation='relu'),
        layers.MaxPooling2D(pool_size=(2, 2)),
        layers.Flatten(),
        layers.Dense(dense_units, activation='relu'),
        layers.Dense(10, activation='softmax')
    ])
    return model


**Compile the Model**

In [4]:
def compile_model(model, learning_rate):
    model.compile(
        optimizer=tf.keras.optimizers.Adam(learning_rate=learning_rate),
        loss='categorical_crossentropy',
        metrics=['accuracy']
    )
    return model


**Train and Evaluate the Model**

In [5]:
def train_and_evaluate_model(model, x_train, y_train, x_test, y_test, batch_size, epochs, datagen=None):
    if datagen:
        history = model.fit(datagen.flow(x_train, y_train, batch_size=batch_size),
                            validation_data=(x_test, y_test),
                            epochs=epochs)
    else:
        history = model.fit(x_train, y_train, batch_size=batch_size, epochs=epochs, validation_data=(x_test, y_test))

    test_loss, test_accuracy = model.evaluate(x_test, y_test, verbose=0)
    return test_loss, test_accuracy


**Add Data Augmentation**

In [6]:
def get_data_augmentation():
    datagen = ImageDataGenerator(
        rotation_range=10,
        width_shift_range=0.1,
        height_shift_range=0.1,
        zoom_range=0.1
    )
    datagen.fit(x_train)
    return datagen

datagen = get_data_augmentation()


**Hyperparameter Tuning with Data Augmentation**

In [7]:
best_accuracy = 0
best_params = {}

hyperparams_combinations = [
    {'num_filters1': 32, 'num_filters2': 64, 'kernel_size': (3, 3), 'dense_units': 128, 'learning_rate': 0.001, 'batch_size': 64, 'epochs': 5},
    {'num_filters1': 64, 'num_filters2': 128, 'kernel_size': (3, 3), 'dense_units': 256, 'learning_rate': 0.0005, 'batch_size': 128, 'epochs': 5},
    {'num_filters1': 32, 'num_filters2': 64, 'kernel_size': (5, 5), 'dense_units': 128, 'learning_rate': 0.0001, 'batch_size': 32, 'epochs': 5},
    {'num_filters1': 16, 'num_filters2': 32, 'kernel_size': (3, 3), 'dense_units': 64, 'learning_rate': 0.002, 'batch_size': 64, 'epochs': 5},
    {'num_filters1': 64, 'num_filters2': 128, 'kernel_size': (5, 5), 'dense_units': 128, 'learning_rate': 0.001, 'batch_size': 128, 'epochs': 5},
    {'num_filters1': 32, 'num_filters2': 64, 'kernel_size': (3, 3), 'dense_units': 256, 'learning_rate': 0.0005, 'batch_size': 32, 'epochs': 5},
    {'num_filters1': 16, 'num_filters2': 32, 'kernel_size': (5, 5), 'dense_units': 64, 'learning_rate': 0.0001, 'batch_size': 32, 'epochs': 5},
    {'num_filters1': 32, 'num_filters2': 64, 'kernel_size': (3, 3), 'dense_units': 64, 'learning_rate': 0.002, 'batch_size': 64, 'epochs': 5},
    {'num_filters1': 128, 'num_filters2': 256, 'kernel_size': (3, 3), 'dense_units': 512, 'learning_rate': 0.0001, 'batch_size': 32, 'epochs': 5},
    {'num_filters1': 64, 'num_filters2': 128, 'kernel_size': (3, 3), 'dense_units': 128, 'learning_rate': 0.001, 'batch_size': 64, 'epochs': 5}
]

for params in hyperparams_combinations:
    print(f"Training with parameters: {params}")
    model = build_cnn_model(params['num_filters1'], params['num_filters2'], params['kernel_size'], params['dense_units'])
    model = compile_model(model, params['learning_rate'])
    test_loss, test_accuracy = train_and_evaluate_model(model, x_train, y_train, x_test, y_test, params['batch_size'], params['epochs'], datagen)
    print(f"Accuracy: {test_accuracy:.4f}, Loss: {test_loss:.4f}\n")

    if test_accuracy > best_accuracy:
        best_accuracy = test_accuracy
        best_params = params

print(f"Best Hyperparameters: {best_params}")
print(f"Best Accuracy: {best_accuracy:.4f}")


Training with parameters: {'num_filters1': 32, 'num_filters2': 64, 'kernel_size': (3, 3), 'dense_units': 128, 'learning_rate': 0.001, 'batch_size': 64, 'epochs': 5}


/usr/local/lib/python3.11/dist-packages/keras/src/layers/convolutional/base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Epoch 1/5


/usr/local/lib/python3.11/dist-packages/keras/src/trainers/data_adapters/py_dataset_adapter.py:122: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


938/938 ━━━━━━━━━━━━━━━━━━━━ 26s 22ms/step - accuracy: 0.8060 - loss: 0.5982 - val_accuracy: 0.9822 - val_loss: 0.0555
Epoch 2/5
938/938 ━━━━━━━━━━━━━━━━━━━━ 18s 19ms/step - accuracy: 0.9634 - loss: 0.1176 - val_accuracy: 0.9879 - val_loss: 0.0367
Epoch 3/5
938/938 ━━━━━━━━━━━━━━━━━━━━ 21s 21ms/step - accuracy: 0.9787 - loss: 0.0713 - val_accuracy: 0.9898 - val_loss: 0.0305
Epoch 4/5
938/938 ━━━━━━━━━━━━━━━━━━━━ 19s 19ms/step - accuracy: 0.9822 - loss: 0.0601 - val_accuracy: 0.9919 - val_loss: 0.0267
Epoch 5/5
938/938 ━━━━━━━━━━━━━━━━━━━━ 21s 20ms/step - accuracy: 0.9827 - loss: 0.0559 - val_accuracy: 0.9919 - val_loss: 0.0259
Accuracy: 0.9919, Loss: 0.0259

Training with parameters: {'num_filters1': 64, 'num_filters2': 128, 'kernel_size': (3, 3), 'dense_units': 256, 'learning_rate': 0.0005, 'batch_size': 128, 'epochs': 5}
Epoch 1/5


/usr/local/lib/python3.11/dist-packages/keras/src/layers/convolutional/base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


469/469 ━━━━━━━━━━━━━━━━━━━━ 24s 45ms/step - accuracy: 0.7619 - loss: 0.7597 - val_accuracy: 0.9796 - val_loss: 0.0644
Epoch 2/5
469/469 ━━━━━━━━━━━━━━━━━━━━ 37s 40ms/step - accuracy: 0.9607 - loss: 0.1291 - val_accuracy: 0.9871 - val_loss: 0.0398
Epoch 3/5
469/469 ━━━━━━━━━━━━━━━━━━━━ 18s 39ms/step - accuracy: 0.9707 - loss: 0.0944 - val_accuracy: 0.9882 - val_loss: 0.0326
Epoch 4/5
469/469 ━━━━━━━━━━━━━━━━━━━━ 21s 39ms/step - accuracy: 0.9785 - loss: 0.0695 - val_accuracy: 0.9900 - val_loss: 0.0326
Epoch 5/5
469/469 ━━━━━━━━━━━━━━━━━━━━ 17s 36ms/step - accuracy: 0.9808 - loss: 0.0600 - val_accuracy: 0.9912 - val_loss: 0.0259
Accuracy: 0.9912, Loss: 0.0259

Training with parameters: {'num_filters1': 32, 'num_filters2': 64, 'kernel_size': (5, 5), 'dense_units': 128, 'learning_rate': 0.0001, 'batch_size': 32, 'epochs': 5}
Epoch 1/5
1875/1875 ━━━━━━━━━━━━━━━━━━━━ 24s 11ms/step - accuracy: 0.6897 - loss: 1.0199 - val_accuracy: 0.9683 - val_loss: 0.1047
Epoch 2/5
1875/1875 ━━━━━━━━━━━━━━━━

**Visualize the Model Summary for Best Hyperparameters**

In [8]:
# Re-train the model with the best hyperparameters for summary and evaluation
best_model = build_cnn_model(
    best_params['num_filters1'],
    best_params['num_filters2'],
    best_params['kernel_size'],
    best_params['dense_units']
)
best_model = compile_model(best_model, best_params['learning_rate'])
train_and_evaluate_model(
    best_model, x_train, y_train, x_test, y_test, best_params['batch_size'], best_params['epochs'], datagen
)

# Display the model summary
best_model.summary()


Epoch 1/5
938/938 ━━━━━━━━━━━━━━━━━━━━ 22s 21ms/step - accuracy: 0.8225 - loss: 0.5514 - val_accuracy: 0.9790 - val_loss: 0.0619
Epoch 2/5
938/938 ━━━━━━━━━━━━━━━━━━━━ 18s 19ms/step - accuracy: 0.9704 - loss: 0.0970 - val_accuracy: 0.9898 - val_loss: 0.0335
Epoch 3/5
938/938 ━━━━━━━━━━━━━━━━━━━━ 22s 20ms/step - accuracy: 0.9792 - loss: 0.0655 - val_accuracy: 0.9920 - val_loss: 0.0259
Epoch 4/5
938/938 ━━━━━━━━━━━━━━━━━━━━ 18s 19ms/step - accuracy: 0.9834 - loss: 0.0539 - val_accuracy: 0.9923 - val_loss: 0.0222
Epoch 5/5
938/938 ━━━━━━━━━━━━━━━━━━━━ 21s 20ms/step - accuracy: 0.9856 - loss: 0.0449 - val_accuracy: 0.9914 - val_loss: 0.0275


Model: "sequential_10"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ conv2d_20 (Conv2D)                   │ (None, 26, 26, 64)          │             640 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ max_pooling2d_20 (MaxPooling2D)      │ (None, 13, 13, 64)          │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ conv2d_21 (Conv2D)                   │ (None, 11, 11, 128)         │          73,856 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ max_pooling2d_21 (MaxPooling2D)      │ (None, 5, 5, 128)           │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ flatten_10 (Flatten)                 │ (None, 3200)                │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_20 (Dense)                     │ (None, 128)                 │         409,728 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_21 (Dense)                     │ (None, 10)                  │           1,290 │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 1,456,544 (5.56 MB)

 Trainable params: 485,514 (1.85 MB)

 Non-trainable params: 0 (0.00 B)

 Optimizer params: 971,030 (3.70 MB)

In [11]:
best_model.save('mnist_3.h5')